# Projet Machine Learning : Xente Fraud Detection
Auteurs : Théo Engels, Hadrien Godts

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline as skPipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE, SMOTENC
from CustomTransformers import StringCleanTransformer, DayTimeTransformer, DropperTransformer, SignTransformer, OHTransformer, FloatTransformer, biningTransformer, weekdayTransformer
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

## Introduction
Ce rapport présente les processus et les résultats de notre travail sur le Xente fraud detection challenge. Nous commençons par une analyse des données présentes dans le dataset, suivie de la hase de feature engineering. Nous présentons ensuite les différents modèles que nous avons entrainés et leurs performances, et nous terminons par une courte conclusion et quelques perspectives de recherche.

L’objectif de ce challenge est de prédire si une transaction bancaire est frauduleuse ou pas, sur la base des différents features contenus dans le dataset.


## Analyse des données

La première chose importante à noter est que le training set contient 95 662 entrées, parmi lesquelles on compte 95 469 transactions légales et 193 frauduleuses : c’est un déséquilibre fort et important à noter car le manque de transactions frauduleuses risque de compliquer l’entrainement des modèles. Nous pallions cette faiblesse dans le feature engineering.

In [ ]:
# Code pour nbr transactions

La seconde chose importante à propos des données est qu’il ne manque aucune donnée : toutes les entrées ont des données pour chaque feature.

In [ ]:
# Code pour données manquantes

Voyons ensuite les différents feature du dataset. S’il y a des choses importantes, nous les mentionnons ici.
-	TransactionId : numéro de la transaction. Cette donnée est à garder car elle permet de soumettre les résultats au défi, mais nous retirerons cette colonne du dataset dans la mesure où le numéro de la transaction ne devrait rien nous apprendre de particulier sur son caractère frauduleux.
-	BatchId : le numéro de groupe de transaction. Comme pour le numéro de transaction, cette donnée ne devrait rien nous apprendre sur le caractère frauduleux d’une transaction. Par ailleurs, dans l’hypothèse improbable où toutes les transactions sont regroupées dans le même groupe, nous ferions face à du target leakage puisque le numéro de Batch nous dirait directement quelque chose à propos de la légalité de la transacition.
-	AccountId : l’identifiant d’un compte client. Nous conservons cette donnée.
-	SubscriptionId : l’identifiant d’une souscription. D’après la description des features, cette donnée semble liée à un compte client, sans autre explication particulièrement révélatrice. Vu ce lien et ce manque de clarté, nous décidons de laisser cette variable hors du dataset, afin d’en réduire la taille.
-	CustomerId : l’identifiant d’un client. À priori, un client peut disposer de plusieurs comptes, donc pour réduire la taille du dataset, nous laissons cette variable hors du dataset en privilégiant AccountId. Cela étant, si on peut établir qu’un compte fraude plus que les autres, il sera pertinent de reconstruire le lien vers le CustomerId.
-	CurrencyCode : la monnaie de la transaction. Il n’y a qu’une seule valeur pour ce feature, donc nous le laissons également hors du dataset.
-	CountryCode : le code du pays de la transaction. Idem que CurrencyCode, nous le laissons donc également hors du dataset.
-	ProviderId KEEP : indique la source du produit acheté via la transaction. Nous conservons cette feature en faisant un one-hot encoding sur les six valeurs possibles, en partant du principe qu'il peut y avoir un lien entre la source du produit et le caractère frauduleux de la transaction.
-	ProductId : donne le code du produit qui a été acheté lors de la transaction. Cette colonne contient 23 valeurs possibles et ne se prête donc pas à du one-hot encoding. En outre, nous disposons de la variable ProductCategory (voir ci-après) donc nous décidons de laisser celle-ci hors du dataset. 
-	ProductCategory : indique la catégorie de produit qui a été fait l’objet de la transaction. Il y a 9 catégories possibles, donc nous pouvons faire du one-hot encoding pour conserver cette feature.
-	ChannelId : indique la méthode de paiement utilisée pour effectuer la transaction. Cette feature nous semble importante, nous la conservons donc en faisant du one-hot encoding une fois de plus.
-	Amount : indique le montant de la transaction, avec son signe (+ ou - pour crédit ou débit). La feature value nous donne déjà le montant absolu, donc nous faisons donc d'abord du binary encoding pour que les montant positif soient désormais true, et les montant négatifs false.
-	Value : indique le montant de la transaction, sans son signe. Nous conservons cette feature.
-	TransactionStartTime : indique la date et l'horaire de la transaction. Nous convertissons le format de cette donnée pour avoir un ordre chronologique, puis nous la séparons en une colonne date et une colonne heure. La colonne date est convertie pour avoir le jour de la semaine et one-hot encodée ensuite. La colonne heure laissée telle quelle pour le moment.
-	PricingStrategy : indique la strucure de prix de Xente pour ses différents marchands. Sans certitude sur ce que cette colonne représente exactement, nous la one-hot encodons et verrons par la suite si elle a un impact ou non.
-	FraudResult : indique si la transaction est frauduleuse. Il s’agit de notre target, nous la gardons donc.

In [ ]:
# Code pour nunique

## Feature engineering

Le dataset est déjà séparé en training et en validation, ce qui nous épargne cette étape. Afin d’organiser efficacement notre travail, nous rédigeons le code sous forme de pipelines.

Nous créons deux préprocesseurs : le premier pour la transformation des données, et le second pour un échantillonnage artificiel. Ce dernier ne peut pas être appliqué au validation set, cela créerait du data leakage – mais il faut toutefois transformer ces données, d’où l’intérêt de séparer les deux préprocesseurs.
Les transformations du premier préprocesseur sont décrites dans le tableau suivant :

| Transformateur | Description |
| --- | --- |
| shuffle | Randomise l’ordre des entrées dans le dataset. |
| remove_target | Retire la colonne « transaction_id » en la sauvegardant pour poster les données sur le site web. Cette étape doit se faire après le shuffle pour que les prédictions soient dans le même ordre que les numéros de transaction. |
| clean_strings | Pour les colonnes qui ont un id précédé d'un string (par exemple : BatchID_578), retire le texte en ne conserve que le numéro |
| refactor_daytime | Convertit le format de TransactionStartTime en format yyyymmddhhmmss. |
| drop | Supprime les features marqués comme non-pertinents (voir analyse des données) |
| convert_amount | Remplace les valeurs de la colonne amount par true si elles sont positives et false si elles sont négatives ; la colonne est renommée « sign ». |
| one_hot_encode | Effectue un one-hot-encoding pour les colonnes suivantes : ProviderId, ProductCategory, ChannelId, PricingStrategy et Sign. |	
| bin |  | 


In [ ]:
# Code pour le preprocessing

## Modèles
Nous entrainons les données adaptées sur trois modèles : un Random Forest, un XGBoost et un MLP ; le tableau Y montre les performances de ces modèles, sans adaptation particulière de leurs paramètres.


In [ ]:
# Code pour le modèle de base

Les résultats sont les suivants :

| Modèle | Public score | Private score |
| --- | --- | --- |
| Random Forest | 0.9994 | 0.9994 |
| XGBoost | 0.75 | 0.73 |
| MLP | 0.9994 | 0.9994 |

![Random Forest Result]()
![XGBoost Result]()
![MLP Result]()

Nous voyons clairement que le XGBoost présente les meilleurs résultats. Nous nous tentons donc de partir de cette base pour améliorer encore le modèle. Nous travaillons principalement sur les paramètres suivants  :
-	Le type de booster
-	n_estimators
-	learning rate
-	max_depth



In [ ]:
# Code pour le modèle avec paramètres améliorés

Voici les résultats obtenus :

| Adaptation | Public score | Private score |
| --- | --- | --- |


Par ailleurs, nous faisons les essais suivants :
-	Effectuer l’échantillonnage artificiel avant le feature engineering dans le training set
-	Conserver le feature « ProductId »

In [ ]:
# Code pour le modèle final

## Conclusion et perspectives
[Conclusion]

Le score obtenu nous semble satisfaisant, d'autant plus qu'il nous place parmi les 128 meilleurs scores du concours, mais nous pensons qu’il est possible de l’améliorer encore. Nous avons plusieurs pistes pour y parvenir :
-	Faire du binning sur l'heure pour avoir matin - journée - soir au lieu de l'heure exacte.

-	L1 & L2
-	subsample
-	min_child_weight
-	gamma

-	Faire davantage de feature engineering avec davantage de domaine knowledge. Au regard des résultats obtenus par les participants au concours, il doit y avoir une manipulation de données qui permet d'améliorer nettement le score, mais à ce stade-ci nous manquons probablement de connaissances spécifiques pour la trouver.
